In [1]:
import tensorflow as tf
import numpy as np

In [3]:

K = 3 # three variables
N = 4 # four gray levels.
n_highest_order = 3
moments = np.array([0, 0, 1]) 
correlation = np.array([0.8, 0.7]) # This would be changed into moments and covariance later on...
variance = moments[2] - moments[0]**2
covariance = correlation * variance
moment_covariance = covariance + moments[0]**2


# K = 3 # three variables
# N = 4 # four gray levels.
# n_highest_order = 3
# moments = [0, 0, 1]
# correlation = [0.8, 0.7] # This would be changed into moments and covariance later on...
# variance = moments[2] - moments[0]**2
# covariance = correlation * variance
# moment_covariance = covariance + moments[0]**2






In [18]:
# define the variable
tf.reset_default_graph()

gray_level = tf.constant([-1, 0, 1, 2], dtype = tf.float32)
gray_level = tf.expand_dims(gray_level, axis = 1)
# depend on how many K. how could you make it work? not sure...
X = tf.meshgrid(gray_level, gray_level, gray_level, indexing = 'ij')


In [19]:
with tf.variable_scope('Lagrange_Multipliers'):
    lambda_sum = tf.get_variable('0', shape=1, initializer=tf.zeros_initializer)
    lambda_marginal = tf.get_variable('1', shape=[K, n_highest_order], initializer=tf.truncated_normal_initializer(stddev = 0.01))
    lambda_covariance = tf.get_variable('2_2', shape= K - 1, initializer=tf.truncated_normal_initializer(stddev = 0.01))

with tf.name_scope('probability_utils'):
    marginal_term = tf.get_variable('marginal_term', shape = N * np.ones(K), initializer=tf.zeros_initializer())
    covariance_term = tf.get_variable('covariance_term', shape = N * np.ones(K), initializer=tf.zeros_initializer)
    for ii in range(K):
        marginal_term = marginal_term + tf.multiply(lambda_marginal[ii,0], X[ii]) + \
        tf.multiply(lambda_marginal[ii,1], tf.square(X[ii])) + tf.multiply(lambda_marginal[ii,2], tf.pow(X[ii], 3))         
    # covariance matrix is different.
    for ii in range(K):
        for dd in range(1, K-ii):
            covariance_X =  tf.multiply(X[ii], X[ii + dd])
            covariance_term = covariance_term + tf.multiply(covariance_X, lambda_covariance[dd - 1])
    constant_term = tf.constant(-1, dtype=tf.float32) + lambda_sum


In [20]:
def marginalize_probability(P, var_name, var_left):
    axis_left = np.nonzero([var not in var_left for var in var_name])
    axis_left = axis_left[0]
    P_marginal = tf.reduce_sum(P, axis = axis_left)
    return P_marginal
with tf.name_scope('probability'):
    P = constant_term + marginal_term + covariance_term
    P_two = {}
    for ii in range(K):
        for dd in range(1, K-ii):
            P_two[(ii, ii + dd)] = marginalize_probability(P, np.array(range(K)), [ii, ii + dd])
    P_one = {}
    P_one[0] = marginalize_probability(P_two[(0, 1)], [0,1], [0])
    for ii in range(1, K):
        P_one[ii] = marginalize_probability(P_two[(0, ii)],[0, ii], [ii])

In [56]:
with tf.name_scope('constraint'):
    sum_constraint= tf.reduce_sum(P_one[0], axis = 0) - 1
    
    # get the first order first.
    marginal_constraint = tf.get_variable('marginal_constraint8', shape=[n_highest_order, K], initializer=tf.zeros_initializer())
    for order in range(n_highest_order):
        for ii in range(K):
            tf.scatter_update(marginal_constraint, [], \
                             tf.squeeze(tf.matmul(tf.expand_dims(P_one[ii], axis = 0),  tf.pow(gray_level, order + 1))) \
                              - moments[order])
    covariance_constraint = tf.get_variable('covariance_contraint', shape=[K, K], initializer=tf.zeros_initializer())
    for ii in range(K):
        for dd in range(1, K-ii):
            tf.scatter_nd_update(covariance_constraint, [ii, ii + dd], \
                                tf.matmul(tf.transpose(gray_level), tf.matmul(P_two[ii, ii+dd], gray_level))\
                                - moment_covariance[dd - 1])

ValueError: Shapes must be equal rank, but are 0 and 1 for 'constraint_15/ScatterUpdate' (op: 'ScatterUpdate') with input shapes: [3,3], [], [].

In [54]:
order = 0
ii = 1
tf.squeeze(tf.matmul(tf.expand_dims(P_one[ii], axis = 0),  tf.pow(gray_level, order + 1))) 


<tf.Tensor 'Squeeze_2:0' shape=() dtype=float32>

In [55]:
moments[order]

0